# When / Otherwise

- This requirement is similar to the last, but now you want to add multiple values based on the voter's position. Modify your `voter_df` DataFrame to add a random number to any voting member that is defined as a `Councilmember`. Use `2` for the `Mayor` and `0` for anything other position.

- The `voter_df` Data Frame is defined and available to you. The `pyspark.sql.functions` library is available as `F`. You can use `F.rand()` to generate the random value.

## Instructions

- Add a column to `voter_df` named `random_val` with the results of the `F.rand()` method for any voter with the title `Councilmember`. Set `random_val` to `2` for the `Mayor`. Set any other title to the value `0`.
- Show some of the Data Frame rows, noting whether the clauses worked.
- Use the `.filter` clause to find 0 in `random_val`.

In [1]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [2]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [3]:
import pyspark.sql.functions as F

# Load the CSV file
voter_df = spark.read.format('csv').options(Header=True).load('file:///home/talentum/test-jupyter/P3/M2/SM2/Dataset/DallasCouncilVoters.csv.gz')

# Add a column to voter_df for a voter based on their position
voter_df = voter_df.withColumn('random_val',
                               F.when(voter_df.TITLE == 'Councilmember', 1)
                               .when(voter_df.TITLE=='Mayor', 2)
                              .otherwise(3))

# Show some of the DataFrame rows
voter_df.show()

# Use the .filter() clause with random_val
voter_df.filter('random_val==3').show()

+----------+-------------+-------------------+----------+
|      DATE|        TITLE|         VOTER_NAME|random_val|
+----------+-------------+-------------------+----------+
|02/08/2017|Councilmember|  Jennifer S. Gates|         1|
|02/08/2017|Councilmember| Philip T. Kingston|         1|
|02/08/2017|        Mayor|Michael S. Rawlings|         2|
|02/08/2017|Councilmember|       Adam Medrano|         1|
|02/08/2017|Councilmember|       Casey Thomas|         1|
|02/08/2017|Councilmember|Carolyn King Arnold|         1|
|02/08/2017|Councilmember|       Scott Griggs|         1|
|02/08/2017|Councilmember|   B. Adam  McGough|         1|
|02/08/2017|Councilmember|       Lee Kleinman|         1|
|02/08/2017|Councilmember|      Sandy Greyson|         1|
|02/08/2017|Councilmember|  Jennifer S. Gates|         1|
|02/08/2017|Councilmember| Philip T. Kingston|         1|
|02/08/2017|        Mayor|Michael S. Rawlings|         2|
|02/08/2017|Councilmember|       Adam Medrano|         1|
|02/08/2017|Co

In [4]:
voter_df.select('TITLE').distinct().show(40, truncate=False)

+--------------------------------------------------------------------------------------------+
|TITLE                                                                                       |
+--------------------------------------------------------------------------------------------+
|Mayor                                                                                       |
|Councilmember                                                                               |
|Deputy Mayor Pro Tem                                                                        |
|null                                                                                        |
| 2020]"                                                                                     |
| authorize an  ordinance  approving  and  adopting  the  final  2019 Service   Plan         |
| authorize an  ordinance  approving and adopting the final Service Plan for 2018-19         |
|Vacant                                           

In [5]:
# Add a new column called splits separated on whitespace
voter_df = voter_df.withColumn('splits', F.split(voter_df.VOTER_NAME, '\s+'))

# Create a new column called first_name based on the first item in splits
voter_df = voter_df.withColumn('first_name', voter_df.splits.getItem(0))

# Get the last entry of the splits list and create a column called last_name
voter_df = voter_df.withColumn('last_name', voter_df.splits.getItem(F.size('splits') - 1))

voter_df = voter_df.withColumn('Middle_name',
                               F.when(F.size('splits')>2,voter_df.splits.getItem(1))
                               .otherwise(' '))

voter_df=voter_df.drop('splits','random_val', 'VOTER_NAME')

voter_df.show()

+----------+-------------+----------+---------+-----------+
|      DATE|        TITLE|first_name|last_name|Middle_name|
+----------+-------------+----------+---------+-----------+
|02/08/2017|Councilmember|  Jennifer|    Gates|         S.|
|02/08/2017|Councilmember|    Philip| Kingston|         T.|
|02/08/2017|        Mayor|   Michael| Rawlings|         S.|
|02/08/2017|Councilmember|      Adam|  Medrano|           |
|02/08/2017|Councilmember|     Casey|   Thomas|           |
|02/08/2017|Councilmember|   Carolyn|   Arnold|       King|
|02/08/2017|Councilmember|     Scott|   Griggs|           |
|02/08/2017|Councilmember|        B.|  McGough|       Adam|
|02/08/2017|Councilmember|       Lee| Kleinman|           |
|02/08/2017|Councilmember|     Sandy|  Greyson|           |
|02/08/2017|Councilmember|  Jennifer|    Gates|         S.|
|02/08/2017|Councilmember|    Philip| Kingston|         T.|
|02/08/2017|        Mayor|   Michael| Rawlings|         S.|
|02/08/2017|Councilmember|      Adam|  M

In [ ]:
# Add a new column called splits separated on whitespace
voter_df = voter_df.withColumn('splits', F.split(voter_df.VOTER_NAME, '\s+'))

# Create a new column called first_name based on the first item in splits
voter_df = voter_df.withColumn('first_name', voter_df.splits.getItem(0))

# Get the last entry of the splits list and create a column called last_name
voter_df = voter_df.withColumn('last_name', voter_df.splits.getItem(F.size('splits') - 1))

voter_df = voter_df.withColumn('Middle_name',
                               F.when(F.size('splits')>2,voter_df.splits.getItem(1))
                               .otherwise(' '))

voter_df=voter_df.drop('splits','random_val', 'VOTER_NAME')

voter_df.show()